In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import numpy as np
import re
import plotly as pl 
import matplotlib.pyplot as plt



In [2]:
df_chocolate = pd.read_csv("clean_flavors_of_cacao.csv", encoding='utf-8')
df_chocolate.head()

,Company,Bean_Origin_or_Bar_Name,REF,Review_Date,Cocoa_Percent,Company_Location,Rating,Bean_Type,Broad_Bean_Origin,Ingredients,Most_Memorable_Characteristics,country_code,latitude,longitude,continent
0,A. Morin,Agua Grande,1876,2016,63.0,France,3.75,missing,Sao Tome & Principe,"4- B,S,C,L","sweet, chocolatey, vegetal",ST,0.186360,6.613081,Africa
1,A. Morin,Kpime,1676,2015,70.0,France,2.75,missing,Togo,"4- B,S,C,L","burnt wood, earthy, choco",TG,8.619543,0.824782,Africa
2,A. Morin,Atsane,1676,2015,70.0,France,3.00,missing,Togo,"4- B,S,C,L","roasty, acidic, nutty",TG,8.619543,0.824782,Africa
3,A. Morin,Akata,1680,2015,70.0,France,3.50,missing,Togo,"4- B,S,C,L","mild profile, chocolaty, spice",TG,8.619543,0.824782,Africa
4,A. Morin,Quilla,1704,2015,70.0,France,3.50,missing,Peru,"4- B,S,C,L","grainy texture, cocoa, sweet",PE,-9.189967,-75.015152,South America


# Machine Learning 



In [3]:
def bin_ratings(rating):
    if rating == 5.75: return 1
    if rating == 5.50: return 1
    if rating == 5.25: return 1
    if rating == 5.00: return 1
    
    if rating == 4.75: return 1
    if rating == 4.50: return 1
    if rating == 4.25: return 1
    if rating == 4.00: return 1
    
    if rating == 3.75: return 1
    if rating == 3.50: return 0
    if rating == 3.25: return 0
    if rating == 3.00: return 0

    if rating == 2.75: return 0
    if rating == 2.50: return 0
    if rating == 2.25: return 0
    if rating == 2.00: return 0
    
    if rating == 1.75: return 0
    if rating == 1.50: return 0
    if rating == 1.25: return 0
    if rating == 1.00: return 0
    
    #print( f"error: rating={rating} type={type(rating)}" )
    return "2"

In [4]:
df_chocolate['Rating'] = df_chocolate['Rating'].apply(bin_ratings)


In [5]:
def to_string(value):
    other = f"_{value}_"
    return other

In [6]:
def reduce_count_vals(df, colname, threshold):
    counts = df[colname].value_counts()
    replace_list = list(counts[counts < threshold].index)

    # Replace in dataframe
    for item in replace_list:
       df[colname] = df[colname].replace(item,"Other")

In [7]:
df_chocolate['Review_Date']= df_chocolate['Review_Date'].apply(to_string)
reduce_count_vals(df_chocolate, 'Review_Date', 100)

## Neural Network!

In [8]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df_chocolate= df_chocolate.drop(columns=["Company","REF","Bean_Origin_or_Bar_Name","Most_Memorable_Characteristics","continent"], axis=1)
df_chocolate.head()

,Review_Date,Cocoa_Percent,Company_Location,Rating,Bean_Type,Broad_Bean_Origin,Ingredients,country_code,latitude,longitude
0,_2016_,63.0,France,1,missing,Sao Tome & Principe,"4- B,S,C,L",ST,0.186360,6.613081
1,_2015_,70.0,France,0,missing,Togo,"4- B,S,C,L",TG,8.619543,0.824782
2,_2015_,70.0,France,0,missing,Togo,"4- B,S,C,L",TG,8.619543,0.824782
3,_2015_,70.0,France,0,missing,Togo,"4- B,S,C,L",TG,8.619543,0.824782
4,_2015_,70.0,France,0,missing,Peru,"4- B,S,C,L",PE,-9.189967,-75.015152


In [9]:
# Generate our categorical variable lists
chocolate_cat = df_chocolate.dtypes[df_chocolate.dtypes == 'object'].index.tolist()
chocolate_cat

['Review_Date',
 'Company_Location',
 'Bean_Type',
 'Broad_Bean_Origin',
 'Ingredients',
 'country_code']

In [10]:


# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_chocolate[chocolate_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(chocolate_cat)
encode_df.head()



,Review_Date_Other,Review_Date__2008_,Review_Date__2009_,Review_Date__2010_,Review_Date__2011_,Review_Date__2012_,Review_Date__2013_,Review_Date__2014_,Review_Date__2015_,Review_Date__2016_,...,country_code_TG,country_code_TT,country_code_TZ,country_code_UG,country_code_US,country_code_VE,country_code_VN,country_code_VU,country_code_WS,country_code_ZZ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Merge one-hot encoded features and drop the originals
df_chocolate = df_chocolate.merge(encode_df, left_index=True, right_index=True).drop(chocolate_cat, 1)
df_chocolate.head()

,Cocoa_Percent,Rating,latitude,longitude,Review_Date_Other,Review_Date__2008_,Review_Date__2009_,Review_Date__2010_,Review_Date__2011_,Review_Date__2012_,...,country_code_TG,country_code_TT,country_code_TZ,country_code_UG,country_code_US,country_code_VE,country_code_VN,country_code_VU,country_code_WS,country_code_ZZ
0,63.0,1,0.186360,6.613081,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,70.0,0,8.619543,0.824782,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,70.0,0,8.619543,0.824782,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,70.0,0,8.619543,0.824782,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,70.0,0,-9.189967,-75.015152,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Split our preprocessed data into our features and target arrays
y = df_chocolate["Rating"]
X = df_chocolate.drop(["Rating"],1) 

# Split the preprocessed data into a training and testing dataset
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

X_train, X_test, y_train, y_test = train_test_split(X,
   y,test_size= 0.4,train_size=0.6 , random_state=1, stratify=y)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
len(X_train_scaled[0])

212

In [15]:
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)


# Random Forest

In [16]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_resampled, y_resampled)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")



 Random forest predictive accuracy: 0.858


In [17]:
# Calculated the balanced accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8584183673469388

In [18]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,593,61
Actual low_risk,50,80


In [19]:
# Print the imbalanced classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91       654
           1       0.57      0.62      0.59       130

    accuracy                           0.86       784
   macro avg       0.74      0.76      0.75       784
weighted avg       0.86      0.86      0.86       784



In [20]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_



In [21]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.16540078518706983, 'Cocoa_Percent'),
 (0.04174798784110652, 'Company_Location_United States of America'),
 (0.04041893382681975, 'longitude'),
 (0.03660528105632691, 'latitude'),
 (0.030921519355748772, 'Ingredients_3- B,S,C'),
 (0.029452602187631124, 'Bean_Type_missing'),
 (0.02920731998234425, 'Review_Date__2015_'),
 (0.02741009667245025, 'Review_Date__2009_'),
 (0.0267180245741858, 'Ingredients_2- B,S'),
 (0.02508045461026639, 'Review_Date__2014_'),
 (0.023034308109295444, 'Review_Date__2016_'),
 (0.02110416772881794, 'Company_Location_Canada'),
 (0.019283740393615457, 'Ingredients_Unknown'),
 (0.019075205979600055, 'Ingredients_4- B,S,C,V'),
 (0.0186392376562422, 'Review_Date__2012_'),
 (0.01855931109842744, 'Review_Date__2013_'),
 (0.01639805106976491, 'Review_Date__2011_'),
 (0.015411730899734525, 'Review_Date_Other'),
 (0.01461879561666939, 'Ingredients_5- B,S,C,V,L'),
 (0.014326245474192879, 'Bean_Type_Trinitario'),
 (0.014032325717544213, 'Ingredients_4- B,S,C,L'),
 (0.013

# Support Vector Machine

In [22]:
# Create the SVM model
svm = SVC(kernel='rbf')

# Train the model
svm.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 SVM model accuracy: 0.838


In [23]:
# Calculated the balanced accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8380102040816326

In [24]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,653,1
Actual low_risk,126,4


In [25]:
# Print the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      1.00      0.91       654
           1       0.80      0.03      0.06       130

    accuracy                           0.84       784
   macro avg       0.82      0.51      0.49       784
weighted avg       0.83      0.84      0.77       784



# Deep Learning Model

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  256
hidden_nodes_layer2 = 128
hidden_nodes_layer3 = 64
hidden_nodes_layer4 = 32
hidden_nodes_layer5 = 16
hidden_nodes_layer6 = 8
hidden_nodes_layer7 = 4


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = 'relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer2, activation = 'relu'))

# other hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer3, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer4, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer5, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer6, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer7, activation = 'relu'))


# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               54528     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 3

In [27]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [28]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics= ['accuracy'])

In [29]:
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [30]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs = 100, callbacks=[cp_callback])

Epoch 1/100
37/37 [==============================] - 1s 2ms/step - loss: 0.4821 - accuracy: 0.8282
Epoch 2/100
37/37 [==============================] - 0s 2ms/step - loss: 0.4198 - accuracy: 0.8333
Epoch 3/100
37/37 [==============================] - 0s 2ms/step - loss: 0.3793 - accuracy: 0.8333
Epoch 4/100
37/37 [==============================] - 0s 2ms/step - loss: 0.3460 - accuracy: 0.8333
Epoch 5/100
37/37 [==============================] - 0s 2ms/step - loss: 0.3185 - accuracy: 0.8333

Epoch 00005: saving model to checkpoints/weights.05.hdf5
Epoch 6/100
37/37 [==============================] - 0s 2ms/step - loss: 0.2915 - accuracy: 0.8410
Epoch 7/100
37/37 [==============================] - 0s 2ms/step - loss: 0.2627 - accuracy: 0.8741
Epoch 8/100
37/37 [==============================] - 0s 2ms/step - loss: 0.2277 - accuracy: 0.8988
Epoch 9/100
37/37 [==============================] - 0s 2ms/step - loss: 0.2024 - accuracy: 0.9014
Epoch 10/100
37/37 [==============================]

37/37 [==============================] - 0s 2ms/step - loss: 0.0577 - accuracy: 0.9745
Epoch 75/100
37/37 [==============================] - 0s 2ms/step - loss: 0.0529 - accuracy: 0.9719

Epoch 00075: saving model to checkpoints/weights.75.hdf5
Epoch 76/100
37/37 [==============================] - 0s 2ms/step - loss: 0.0498 - accuracy: 0.9694
Epoch 77/100
37/37 [==============================] - 0s 2ms/step - loss: 0.0505 - accuracy: 0.9711
Epoch 78/100
37/37 [==============================] - 0s 2ms/step - loss: 0.0683 - accuracy: 0.9685
Epoch 79/100
37/37 [==============================] - 0s 3ms/step - loss: 0.0404 - accuracy: 0.9787
Epoch 80/100
37/37 [==============================] - 0s 2ms/step - loss: 0.0416 - accuracy: 0.9728

Epoch 00080: saving model to checkpoints/weights.80.hdf5
Epoch 81/100
37/37 [==============================] - 0s 2ms/step - loss: 0.0387 - accuracy: 0.9728
Epoch 82/100
37/37 [==============================] - 0s 2ms/step - loss: 0.0391 - accuracy: 0.97

In [31]:


# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")



25/25 - 0s - loss: 2.8068 - accuracy: 0.7985
Loss: 2.8067543506622314, Accuracy: 0.7984693646430969


In [ ]:
nn.save("Chocolate_Ratings_ML.h5")